<a href="https://colab.research.google.com/github/ELiTE0005/DeepLearningTechniques/blob/main/exp_13_pre_trained_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip install -q transformers datasets accelerate evaluate

# Imports
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate, torch

# Load small sentiment dataset (SST2)
dataset = load_dataset("glue", "sst2")
train_data = dataset["train"].select(range(200))
val_data = dataset["validation"].select(range(100))

# Load tokenizer + model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize data
def tokenize(batch): return tokenizer(batch["sentence"], truncation=True, padding="max_length", max_length=128)
train_enc = train_data.map(tokenize, batched=True)
val_enc = val_data.map(tokenize, batched=True)
train_enc.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_enc.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Define accuracy metric
metric = evaluate.load("accuracy")
def compute_metrics(p): preds = p.predictions.argmax(-1); return metric.compute(predictions=preds, references=p.label_ids)

# Training setup
args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    eval_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    fp16=torch.cuda.is_available()
)

# Train
trainer = Trainer(model=model, args=args, train_dataset=train_enc, eval_dataset=val_enc,
                  tokenizer=tokenizer, compute_metrics=compute_metrics)
trainer.train()

# Evaluate
print(trainer.evaluate())

# Quick inference
text = "This movie was amazing!"
inputs = tokenizer(text, return_tensors="pt").to(model.device)
pred = torch.argmax(model(**inputs).logits, dim=-1).item()
print(f"\nText: {text}\nSentiment: {'positive' if pred==1 else 'negative'}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: elite0007 (elite0007-srm-institute-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.694800,0.689474,0.500000


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.68947434425354, 'eval_accuracy': 0.5, 'eval_runtime': 24.1372, 'eval_samples_per_second': 4.143, 'eval_steps_per_second': 0.539, 'epoch': 1.0}

Text: This movie was amazing!
Sentiment: negative
